In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
import math 

In [3]:
from sklearn.model_selection import train_test_split

# Carga de datos 
Se cargan los datos y se hace un pequeño analisis de la correlacion y varianza de estos datos

In [19]:
train_url = "./modelsData/trafficWeatherAccident.csv"
data = pd.read_csv(train_url , header = 0)

In [34]:
data.head

<bound method NDFrame.head of                   zona              subZona  velocidadProm  tiempoProm  \
0                bronx   University Heights      37.330833  119.666667   
1               queens          Bay Terrace      46.391667   47.583333   
2            manhattan   Washington Heights      55.296667  133.750000   
3             brooklyn        Fort Hamilton      46.236667   45.833333   
4            manhattan   Financial District      19.465833  388.958333   
5               queens     South Ozone Park      26.765000  410.250000   
6               queens         Forest Hills      45.355833  196.500000   
7        staten island          Shore Acres      40.590833   52.333333   
8        staten island         Graniteville      39.554583  159.791667   
9        staten island          Willowbrook      32.151667   47.250000   
10              queens           Beechhurst      42.040972  415.986111   
11              queens              Astoria      26.585707  328.777778   
12      

# Tecnicas de conversion de descriptores 
Se busca generealizar el peso de los descriptores en el sentido de que los descriptores tales como distrito, barrios, calle y decripcion de clima con la tenica de **One Hot Encoding**, en el caso de la **hora** no se generalizo con esta tecnica por problemas de **series de tiempo** por lo que se usó Sin.


In [20]:
dfDummies = pd.get_dummies(data['zona'], prefix = 'zona')
dfDummies1 = pd.get_dummies(data['subZona'], prefix = 'subZona')
dfDummies2 = np.sin(data['hora']) 
dfDummies3 = pd.get_dummies(data['linkName'], prefix = 'linkName') 
dfDummies4 = pd.get_dummies(data['descripcionClima'], prefix = 'clima') 

In [21]:
df = pd.concat([data, dfDummies], axis=1)
df = pd.concat([df, dfDummies1], axis=1)
df = pd.concat([df, dfDummies2], axis=1)
df = pd.concat([df, dfDummies3], axis=1)
df = pd.concat([df, dfDummies4], axis=1)

In [22]:
dfPredict = df.drop(["zona","subZona", "hora","descripcionClima","encodedPolyLine","latitude","longitude","linkName","date"], axis = 1)  

Se define la x y el output esperado

In [24]:
X = dfPredict.drop(['accidentes'] , axis = 1)
y = dfPredict['accidentes'] > 0 

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=.2, random_state=42)

In [11]:
X_train.shape , X_test.shape

((1214384, 263), (303596, 263))

In [26]:
X_test.to_csv("./datasetsModels/datasetPrediccionProbabilidadAccidente.csv")

# Modelos

Queremos predecir la probabilidad de que haya un accidente basado en un conjunto de condiciones iniciales, para ellos comprobaremos si maquinas de soporte vectorial, funcionan mejor que una simple regresión logistica, incluso cuando los datos tienen muchas columnas, y son dispersos

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
from sklearn.svm import SVC

In [15]:
clf = LogisticRegression()

In [16]:
clf.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
clf.score(X_train, y_train)

0.9615928734238923

In [18]:
clf.score(X_test, y_test)

0.9612445486765306

In [45]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics


clfKfolding = LogisticRegression()

scores = cross_val_score(clfKfolding,X_train , y_train, cv=6)
print ("cross-validated scores:", scores)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

cross-validated scores: [0.96160535 0.96163006 0.96162987 0.9616101  0.96155081 0.96157058]


In [56]:
clf.predict_proba(X_train.iloc[:100])

array([[4.41981702e-02, 9.55801830e-01],
       [1.62576055e-01, 8.37423945e-01],
       [8.77823743e-02, 9.12217626e-01],
       [6.39943882e-02, 9.36005612e-01],
       [4.08138479e-02, 9.59186152e-01],
       [1.53066532e-02, 9.84693347e-01],
       [1.70872387e-03, 9.98291276e-01],
       [3.64599045e-02, 9.63540096e-01],
       [1.07140448e-04, 9.99892860e-01],
       [3.94880978e-02, 9.60511902e-01],
       [1.08855292e-01, 8.91144708e-01],
       [1.81196986e-02, 9.81880301e-01],
       [6.68283388e-05, 9.99933172e-01],
       [2.16030993e-02, 9.78396901e-01],
       [8.53734866e-02, 9.14626513e-01],
       [6.23557652e-03, 9.93764423e-01],
       [1.06713839e-01, 8.93286161e-01],
       [1.22270755e-04, 9.99877729e-01],
       [6.04979206e-02, 9.39502079e-01],
       [2.34325198e-02, 9.76567480e-01],
       [8.94011287e-02, 9.10598871e-01],
       [2.62299634e-02, 9.73770037e-01],
       [6.29169637e-03, 9.93708304e-01],
       [1.01024435e-01, 8.98975565e-01],
       [3.037013

In [57]:
X_train.iloc[:100]

,velocidadProm,tiempoProm,diaSemana,numeroVehiculos,humedad,presion,temperatura,windDirection,windSpeed,heridos,...,clima_shower rain,clima_sky is clear,clima_smoke,clima_snow,clima_squalls,clima_thunderstorm,clima_thunderstorm with heavy rain,clima_thunderstorm with light rain,clima_thunderstorm with rain,clima_very heavy rain
432780,47.435500,90.291667,4,22,78.0,1028.0,289.951333,231.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
412515,53.691667,103.083333,5,12,40.0,1010.0,302.260000,208.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
1511441,56.513750,110.375000,1,24,99.0,1033.0,287.966667,145.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
771710,55.658333,134.666667,6,12,81.0,1016.0,282.300000,330.0,4.0,0,...,0,0,0,0,0,0,0,0,0,0
943698,27.129583,263.666667,2,24,87.0,1019.0,280.560000,0.0,0.0,0,...,0,1,0,0,0,0,0,0,0,0
187620,35.225278,173.444444,3,36,76.0,1015.0,278.800000,103.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
787370,16.513750,521.166667,7,24,52.0,1045.0,283.529333,68.0,2.0,0,...,0,0,0,0,0,0,0,0,0,0
248218,22.830000,118.083333,2,12,25.0,1024.0,286.450000,0.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
196824,57.160909,80.909091,7,11,69.0,1016.0,300.610000,53.0,1.0,2,...,0,0,0,0,0,0,0,0,0,0
929223,53.692500,139.083333,7,12,44.0,1011.0,299.320000,30.0,3.0,0,...,0,0,1,0,0,0,0,0,0,0
